# 5.6 GPU

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [2]:
torch.cuda.device_count()

0

In [3]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

In [4]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [5]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [6]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.5249, 0.7750, 0.8525],
        [0.5575, 0.5507, 0.2106]])

## M1 Config for d2l

Blog 里指定的 d2l==0.17.6 M 芯片不兼容，去掉版本号。

## Pytorch for Apple Silicon

In [2]:
import torch
torch.__version__
torch.device("mps")

device(type='mps')

In [3]:
a = torch.rand(3,5)
print(a)
b = torch.device('mps')
print(b)

tensor([[0.7821, 0.1719, 0.9009, 0.3457, 0.0928],
        [0.4093, 0.7747, 0.3700, 0.7459, 0.4032],
        [0.7861, 0.9759, 0.1333, 0.1236, 0.3908]])
mps


In [4]:
import torch
import time

# 创建随机矩阵
a = torch.randn(10000, 1000)
b = torch.randn(1000, 10000)

# CPU计算
t0 = time.time()
c = torch.matmul(a, b)
t1 = time.time()
print("CPU Time:", t1 - t0)

# 将数据移动到GPU
device = torch.device('mps')
a = a.to(device)
b = b.to(device)

# GPU计算
t2 = time.time()
c = torch.matmul(a, b)
t3 = time.time()
print("GPU Time:", t3 - t2)

CPU Time: 0.336871862411499
GPU Time: 0.15578484535217285


In [11]:
X = torch.ones(2, 3, device=torch.device('mps'))
X

/opt/homebrew/Caskroom/miniconda/base/envs/d2l/lib/python3.9/site-packages/torch/_tensor_str.py:103: UserWarning: The operator 'aten::bitwise_and.Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(tensor_view, torch.isfinite(tensor_view) & tensor_view.ne(0))


tensor([[1., 1., 1.],
        [1., 1., 1.]], device='mps:0')

In [13]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=torch.device('mps'))

In [14]:
net(X)

tensor([[0.2598],
        [0.2598]], device='mps:0', grad_fn=<MpsLinearBackward0>)

In [15]:
net[0].weight.data.device

device(type='mps', index=0)